In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Function to filter dataset based on allergy and region
def filter_dataset(df, allergy, region):
    return df[(df['Allergy'] != allergy) & (df['Region'] == region)]

# Nutritional target values
target_nutrients = {
    'Proteins': 50,
    'Carbohydrates': 275,
    'Fats': 78,
    'Fiber': 28,
    'Energy(kcal)': 2000
}

# Filtering the dataset based on user preferences
user_allergy = 'egg allergy'  # example allergy
user_region = 'North'         # example region
filtered_data = filter_dataset(nutrition_data, user_allergy, user_region)

# Feature columns to consider for k-NN
features = ['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)', 'Carbon Footprint(kg/CO2)']

# Preparing the data for k-NN
X = filtered_data[features].values
y = filtered_data['Food'].values

# Initializing and fitting k-NN model
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X)

# Target nutrient values as an array
target_values = np.array([target_nutrients['Proteins'], target_nutrients['Carbohydrates'],
                          target_nutrients['Fats'], target_nutrients['Fiber'],
                          target_nutrients['Energy(kcal)'], 0]).reshape(1, -1)

# Finding the k-nearest neighbors
distances, indices = knn.kneighbors(target_values)

# Displaying the recommended food items
recommended_foods = filtered_data.iloc[indices[0]]

recommended_foods[['Food','Energy(kcal)', 'Proteins', 'Carbohydrates', 'Fats', 'Fiber',
                   'Carbon Footprint(kg/CO2)']]


In [1]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = "nutrition_cf - Sheet1.csv"  # Replace with your actual file path
df = pd.read_csv(file_path)

# Function to filter and recommend
def recommend_meals(df, allergies, region, category, target_nutrients):
    # Filter based on category, region, and allergies
    df_filtered = df[
        (df['Category'].str.lower() == category.lower()) &
        (df['Region'].str.lower() == region.lower()) &
        (~df['Allergy'].str.contains('|'.join(allergies), case=False))
    ]

    recommendations = {}
    for meal_type in ['breakfast', 'lunch', 'dinner', 'snacks']:
        # Filter for the specific meal type (you might need to adjust this based on your data)
        df_meal = df_filtered[df_filtered['Type'].str.lower() == meal_type]

        if not df_meal.empty:
            # Calculate distances to target nutrients
            X = df_meal[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']].values
            target_values = np.array([target_nutrients[nutrient] for nutrient in ['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']]).reshape(1, -1)
            distances = np.linalg.norm(X - target_values, axis=1)

            # Recommend the closest item
            recommended_index = np.argmin(distances)
            recommendations[meal_type] = df_meal.iloc[recommended_index]['Food']
        else:
            recommendations[meal_type] = "No recommendations found for this meal type."

    return recommendations

# Example usage
allergies = ['nuts', 'soy']
region = 'south'
category = 'veg'
target_nutrients = {
    'Proteins': 30,  # Adjust target nutrients for each meal if needed
    'Carbohydrates': 100,
    'Fats': 20,
    'Fiber': 10,
    'Energy(kcal)': 500
}

recommendations = recommend_meals(df, allergies, region, category, target_nutrients)

for meal_type, food in recommendations.items():
    print(f"{meal_type.capitalize()}: {food}")

Breakfast: Bajra Dosa
Lunch: Black Rice (pulao)
Dinner: No recommendations found for this meal type.
Snacks: Chana Dal bada


In [11]:
##use this code
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import re

# Load the dataset
file_path = 'nutrition_cf - Sheet1.csv'
nutrition_data = pd.read_csv(file_path)

import re
associative_rules = {
    tuple(df["Associativity"]==1): df["Associativity"]==2,   ##roti : dry sabzi
    #df["Food"]==re.findall("idli|dosa",df["Food"]) : [df["Food"]=="sambhar", df["Food"]=="chutney"],
    #df["Food"]== re.findall("*bread", df["Food"]) : [df["Food"]=="Egg white omelette", df["Food"]=="Egg Sunny Sideup"],
    tuple(df["Food"]=="Hummus") : df["Food"]=="PITA BREAD (WHEAT)",
    tuple(df["Associativity"]==0): df["Associativity"],  #confirm this
    tuple(df["Associativity"]==3): [df["Associativity"]==8, df["Associativity"]==4],    ##
    tuple(df["Type"]=="Drinks") : df["Type"] == "Breakfast",
    tuple(df["Associativity"]==5) : df["Associativity"]==4,
    tuple(df["Associativity"]==6) : df["Associativity"]==7
       
}
if (df["Associativity"]==1):
    print(df["Food"])

# Function to filter dataset based on multiple allergies, region, and category using regex
def filter_dataset(df, allergies, region_pattern, category):
    if allergies != "no-allergies":
        allergy_condition = df['Allergy'].apply(lambda x: not any(re.search(allergy, x, re.IGNORECASE) for allergy in allergies))
    else:
        allergy_condition=df['Allergy']
    region_filter = df['Region'].apply(lambda x: re.search(region_pattern, x, re.IGNORECASE) is not None)
    category_filter = df['Category'].str.lower() == category
    return df[allergy_condition & category_filter & region_filter]

# Nutritional target values
target_breakfast = {
    'Proteins': 12,
    'Carbohydrates': 60,
    'Fats': 12,
    'Fiber': 6,
    'Energy(kcal)': 450
}

target_lunch = {
    'Proteins': 18,
    'Carbohydrates': 90,
    'Fats': 18,
    'Fiber': 9,
    'Energy(kcal)': 650
}

target_snacks = {
    'Proteins': 7,
    'Carbohydrates': 35,
    'Fats': 7,
    'Fiber': 3,
    'Energy(kcal)': 250
}

target_dinner = {
    'Proteins': 17,
    'Carbohydrates': 80,
    'Fats': 13,
    'Fiber': 8,
    'Energy(kcal)': 550
}

# Function to apply associative rules
def apply_associative_rules(filtered_data, associative_rules):
    enhanced_recommendations = set(filtered_data['Food'])
    print(filtered_data)
    for item in filtered_data['Food']:
        if item.lower() in associative_rules:
            enhanced_recommendations.update(associative_rules[item.lower()])
    return filtered_data[filtered_data['Food'].isin(enhanced_recommendations)]

# User inputs for allergies, region, and category
user_allergies_input = input("Enter allergies separated by commas (e.g., mushroom, peanut): ")
user_region_pattern = input("Enter region pattern (regex) (e.g., North): ")
user_category = input("Enter category (e.g., veg, non-veg, eggetarian): ")

# Parsing user allergies input into a list
user_allergies = [allergy.strip() for allergy in user_allergies_input.split(',')]

# Filtering the dataset based on user preferences
filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)
#print(filtered_data)
#print(type(filtered_data))

#Apply Associative Rules
enhanced_neighbors = apply_associative_rules(filtered_data, associative_rules)

# Function to recommend food items for a specific meal type
def recommend_food(df, meal_type, target_nutrients):
    meal_data = df[df['Type'].str.contains(meal_type, case=False, na=False)]
    features = ['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)', 'Carbon Footprint(kg CO2e)']

    X = meal_data[features].values
    y = meal_data['Food'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    knn.fit(X_train_scaled, y_train)

    target_values = np.array([target_nutrients['Proteins'], target_nutrients['Carbohydrates'],
                              target_nutrients['Fats'], target_nutrients['Fiber'],
                              target_nutrients['Energy(kcal)'], 0]).reshape(1, -1)
    target_values_scaled = scaler.transform(target_values)

    distances, indices = knn.kneighbors(target_values_scaled)

    recommended_foods = meal_data.iloc[indices[0]]

    return recommended_foods

'''
# Recommend food items for each meal type
recommended_breakfast = recommend_food(enhanced_neighbors, 'Breakfast', target_breakfast)
recommended_lunch = recommend_food(enhanced_neighbors, 'Lunch', target_lunch)
recommended_dinner = recommend_food(enhanced_neighbors, 'Dinner', target_snacks)
recommended_snacks = recommend_food(enhanced_neighbors, 'Snacks', target_dinner)

# Display recommended food items
print("Recommended Breakfast Foods:")
print(recommended_breakfast[['Food','Allergy']])

print("\nRecommended Lunch Foods:")
print(recommended_lunch[['Food','Allergy']])

print("\nRecommended Dinner Foods:")
print(recommended_dinner[['Food','Allergy']])

print("\nRecommended Snacks Foods:")
print(recommended_snacks[['Food','Allergy']]) '''


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
#Allergy,category and region filtering works
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import re

# Load the dataset
file_path = 'nutrition_data.csv'
nutrition_data = pd.read_csv(file_path)

# Function to filter dataset based on multiple allergies, region, and category using regex
def filter_dataset(df, allergies, region_pattern, category):
    allergy_filter = df['Allergy'].apply(lambda x: not any(re.search(allergy, x, re.IGNORECASE) for allergy in allergies))
    region_filter = df['Region'].apply(lambda x: re.search(region_pattern, x, re.IGNORECASE) is not None)
    category_filter = df['Category'].str.lower() == category
    return df[allergy_filter & category_filter & region_filter]

# Nutritional target values
target_nutrients = {
    'Proteins': 50,
    'Carbohydrates': 275,
    'Fats': 78,
    'Fiber': 28,
    'Energy(kcal)': 2000
}

# User inputs for allergies, region, and category
user_allergies_input = input("Enter allergies separated by commas (e.g., mushroom, peanut): ")
user_region_pattern = input("Enter region pattern (regex) (e.g., North): ")
user_category = input("Enter category (e.g., veg, non-veg, eggetarian): ")

# Parsing user allergies input into a list
user_allergies = [allergy.strip() for allergy in user_allergies_input.split(',')]

# Filtering the dataset based on user preferences
filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)
#print(filtered_data)
# Feature columns to consider for k-NN
features = ['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)', 'Carbon Footprint(kg CO2e)']

# Preparing the data for k-NN
X = filtered_data[features].values
y = filtered_data['Food'].values

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initializing and fitting k-NN classifier
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train_scaled, y_train)

# Target nutrient values as an array
target_values = np.array([target_nutrients['Proteins'], target_nutrients['Carbohydrates'],
                          target_nutrients['Fats'], target_nutrients['Fiber'],
                          target_nutrients['Energy(kcal)'], 0]).reshape(1, -1)
target_values_scaled = scaler.transform(target_values)

# Finding the k-nearest neighbors
distances, indices = knn.kneighbors(target_values_scaled)

# Displaying the recommended food items
recommended_foods = filtered_data.iloc[indices[0]]

print("Recommended Foods:")
print(recommended_foods[['Food','Region', 'Category','Allergy', 'Energy(kcal)', 'Proteins', 'Carbohydrates', 'Fats', 'Fiber',
                         'Carbon Footprint(kg CO2e)']])


In [ ]:
#umang's code 
#umang's basic code(dont touch or i kick ur ass)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import re

# Load the dataset
file_path = 'nutrition_cf - Sheet2.csv'
nutrition_data = pd.read_csv(file_path)

# Function to filter dataset based on multiple allergies, region, and category using regex
def filter_dataset(df, allergies, region_pattern, category):
    if allergies == ["no-allergies"]:
        allergy_condition = True
    else:
        allergy_condition = df['Allergy'].apply(lambda x: not any(re.search(allergy, x, re.IGNORECASE) for allergy in allergies))

    if category == 'veg':
        category_condition = df['Category'].str.lower() == 'veg'
    elif category == 'eggetarian':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg'])
    elif category == 'non-veg':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg', 'non-veg'])
    else:
        category_condition = False

    if region_pattern != "Null":
        region_filter = df['Region'].apply(lambda x: re.search(region_pattern, x, re.IGNORECASE) is not None)
    else:
        return df[allergy_condition & category_condition]

    return df[allergy_condition & region_filter & category_condition]

# Nutritional target values
target_breakfast = {
    'Proteins': 12,
    'Carbohydrates': 60,
    'Fats': 12,
    'Fiber': 6,
    'Energy(kcal)': 450
}

target_lunch = {
    'Proteins': 18,
    'Carbohydrates': 90,
    'Fats': 18,
    'Fiber': 9,
    'Energy(kcal)': 650
}

target_snacks = {
    'Proteins': 7,
    'Carbohydrates': 35,
    'Fats': 7,
    'Fiber': 3,
    'Energy(kcal)': 250
}

target_dinner = {
    'Proteins': 17,
    'Carbohydrates': 80,
    'Fats': 13,
    'Fiber': 8,
    'Energy(kcal)': 550
}

target_appetizers = {
    'Proteins': 5,
    'Carbohydrates': 30,
    'Fats': 5,
    'Fiber': 3,
    'Energy(kcal)': 200
}

# User inputs for allergies, region, and category
user_allergies_input = input("Enter allergies separated by commas (e.g., mushroom, peanut): ")
user_region_pattern = input("Enter region pattern (regex) (e.g., North): ")
user_category = input("Enter category (e.g., veg, non-veg, eggetarian): ")

print("User preference\nAllergy : "+user_allergies_input+"\nRegion : "+user_region_pattern+"\nCategory : "+user_category+"\n")

# Parsing user allergies input into a list
user_allergies = [allergy.strip() for allergy in user_allergies_input.split(',')]
if user_allergies == ['']:
    user_allergies = ["no-allergies"]

def recommend_food(df, meal_type, target_nutrients, num_recommendations=5):
    meal_data = df[df['Type'].str.contains(meal_type, case=False, na=False)]
    features = ['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)', 'Carbon Footprint(kg CO2e)']

    X = meal_data[features].values
    y = meal_data['Food'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    knn = KNeighborsClassifier(n_neighbors=num_recommendations*2, metric='cosine')
    knn.fit(X_train_scaled, y_train)

    target_values = np.array([target_nutrients['Proteins'], target_nutrients['Carbohydrates'],
                              target_nutrients['Fats'], target_nutrients['Fiber'],
                              target_nutrients['Energy(kcal)'], 0]).reshape(1, -1)
    target_values_scaled = scaler.transform(target_values)

    distances, indices = knn.kneighbors(target_values_scaled)

    recommended_foods = meal_data.iloc[indices[0]]
    return recommended_foods.head(num_recommendations)

try:
    # Filtering the dataset based on user preferences
    filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)

    # Function to recommend food items for a specific meal type
    recommended_breakfast = recommend_food(filtered_data, 'Breakfast', target_breakfast)
    recommended_snacks = recommend_food(filtered_data, 'Snacks', target_snacks)
    recommended_appetizers = recommend_food(filtered_data, 'Appetizer', target_appetizers)

except:
    filtered_data = filter_dataset(nutrition_data, user_allergies, "Null", user_category)
    # Function to recommend food items for a specific meal type
    recommended_breakfast = recommend_food(filtered_data, 'Breakfast', target_breakfast)
    recommended_snacks = recommend_food(filtered_data, 'Snacks', target_snacks)
    recommended_appetizers = recommend_food(filtered_data, 'Appetizer', target_appetizers)
finally:
    filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)
    recommended_lunch = recommend_food(filtered_data, 'Lunch', target_lunch)
    recommended_dinner = recommend_food(filtered_data, 'Dinner', target_dinner)

# Define associativity rules
associativity_rules = {
    '1': ['2'],
    '3': ['2', '4', '8'],
    '5': ['4', '8'],
    '6': ['7'],
    # Add more rules as needed
}

# Function to print recommendations with associative rules
def print_recommendations_with_associative_rules(recommended_foods, associative_rules, valid_associations):
    print("Recommended Foods with Associativity:")
    for index, row in recommended_foods.iterrows():
        food_item = row['Food']
        associativity = str(row['Associativity'])

        # Split associativity values if they are combined (e.g., '2,8')
        associativity_values = [value.strip() for value in associativity.split(',')]

        if any(value in valid_associations for value in associativity_values):
            associated_foods = []
            for value in associativity_values:
                if value in associative_rules:
                    associated_food_items = recommended_foods[recommended_foods['Associativity'].isin(associative_rules[value])]
                    if not associated_food_items.empty:
                        associated_foods.append(associated_food_items['Food'].sample(1).values[0])

            if associated_foods:
                print(f"{food_item} (associated with: {', '.join(associated_foods)})")
            else:
                print(f"{food_item}")

# Function to ensure nutritional requirements
def ensure_nutritional_requirements(recommended_foods, target_nutrients):
    total_nutrients = recommended_foods[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']].sum()
    meets_requirements = all(total_nutrients >= pd.Series(target_nutrients))
    return meets_requirements

# Function to print recommendations
def print_recommendations(recommended_foods, meal_type, target_nutrients):
    print(f"\nRecommended {meal_type.capitalize()} Foods:")
    if ensure_nutritional_requirements(recommended_foods, target_nutrients):
        print_recommendations_with_associative_rules(recommended_foods, associativity_rules, ['0', '1', '3', '5', '6'])
    else:
        print(f"No {meal_type} recommendations meet the nutritional requirements.")

# Recommend food items for each meal type
print_recommendations(recommended_breakfast, 'breakfast', target_breakfast)
print_recommendations(recommended_lunch, 'lunch', target_lunch)
print_recommendations(recommended_dinner, 'dinner', target_dinner)
print_recommendations(recommended_snacks, 'snacks', target_snacks)
print_recommendations(recommended_appetizers, 'appetizers', target_appetizers)


In [ ]:
#umang current code
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import re

# Load the dataset
file_path = 'nutrition_cf.csv'
nutrition_data = pd.read_csv(file_path)

# Function to filter dataset based on multiple allergies, region, and category using regex
def filter_dataset(df, allergies, region_pattern, category):
    if allergies == ["no-allergies"]:
        allergy_condition = True
    else:
        allergy_condition = df['Allergy'].apply(lambda x: not any(re.search(allergy, x, re.IGNORECASE) for allergy in allergies))

    if category == 'veg':
        category_condition = df['Category'].str.lower() == 'veg'
    elif category == 'eggetarian':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg'])
    elif category == 'non-veg':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg', 'non-veg'])
    else:
        category_condition = False

    if region_pattern != "Null":
        region_filter = df['Region'].apply(lambda x: re.search(region_pattern, x, re.IGNORECASE) is not None)
    else:
        return df[allergy_condition & category_condition]

    return df[allergy_condition & region_filter & category_condition]

# Nutritional target values
target_breakfast = {
    'Proteins': 12,
    'Carbohydrates': 60,
    'Fats': 12,
    'Fiber': 6,
    'Energy(kcal)': 450
}

target_lunch = {
    'Proteins': 18,
    'Carbohydrates': 90,
    'Fats': 18,
    'Fiber': 9,
    'Energy(kcal)': 650
}

target_snacks = {
    'Proteins': 7,
    'Carbohydrates': 35,
    'Fats': 7,
    'Fiber': 3,
    'Energy(kcal)': 250
}

target_dinner = {
    'Proteins': 17,
    'Carbohydrates': 80,
    'Fats': 13,
    'Fiber': 8,
    'Energy(kcal)': 550
}

target_appetizers = {
    'Proteins': 5,
    'Carbohydrates': 30,
    'Fats': 5,
    'Fiber': 3,
    'Energy(kcal)': 200
}

# User inputs for allergies, region, and category
user_allergies_input = input("Enter allergies separated by commas (e.g., mushroom, peanut): ")
user_region_pattern = input("Enter region pattern (regex) (e.g., North): ")
user_category = input("Enter category (e.g., veg, non-veg, eggetarian): ")

print("User preference\nAllergy : "+user_allergies_input+"\nRegion : "+user_region_pattern+"\nCategory : "+user_category+"\n")

# Parsing user allergies input into a list
user_allergies = [allergy.strip() for allergy in user_allergies_input.split(',')]
if user_allergies == ['']:
    user_allergies = ["no-allergies"]

def recommend_food(df, meal_type, target_nutrients, num_recommendations=5):
    meal_data = df[df['Type'].str.contains(meal_type, case=False, na=False)]
    features = ['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)', 'Carbon Footprint(kg CO2e)']

    X = meal_data[features].values
    y = meal_data['Food'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    knn = KNeighborsClassifier(n_neighbors=num_recommendations*2, metric='cosine')
    knn.fit(X_train_scaled, y_train)

    target_values = np.array([target_nutrients['Proteins'], target_nutrients['Carbohydrates'],
                              target_nutrients['Fats'], target_nutrients['Fiber'],
                              target_nutrients['Energy(kcal)'], 0]).reshape(1, -1)
    target_values_scaled = scaler.transform(target_values)

    distances, indices = knn.kneighbors(target_values_scaled)

    recommended_foods = meal_data.iloc[indices[0]]
    return recommended_foods.head(num_recommendations)

# Filtering the dataset based on user preferences
try:
    # Filtering the dataset based on user preferences
    filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)
    # Function to recommend food items for a specific meal type
    recommended_breakfast = recommend_food(filtered_data, 'Breakfast', target_breakfast)
    recommended_snacks = recommend_food(filtered_data, 'Snacks', target_snacks)
    recommended_appetizers = recommend_food(filtered_data, 'Appetizer', target_appetizers)

except:
    filtered_data = filter_dataset(nutrition_data, user_allergies, "Null", user_category)
    # Function to recommend food items for a specific meal type
    recommended_breakfast = recommend_food(filtered_data, 'Breakfast', target_breakfast)
    recommended_snacks = recommend_food(filtered_data, 'Snacks', target_snacks)
    recommended_appetizers = recommend_food(filtered_data, 'Appetizer', target_appetizers)
finally:
    filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)
    recommended_lunch = recommend_food(filtered_data, 'Lunch', target_lunch)
    recommended_dinner = recommend_food(filtered_data, 'Dinner', target_dinner)

# Define associativity rules
associativity_rules = {
    '1': ['2'],
    '3': ['2', '4', '8'],
    '5': ['4', '8'],
    '6': ['7'],
    # Add more rules as needed
}

# Function to print recommendations with associative rules
def print_recommendations_with_associative_rules(recommended_foods, associative_rules, valid_associations):
    print("Recommended Foods with Associativity:")
    for index, row in recommended_foods.iterrows():
        food_item = row['Food']
        associativity = str(row['Associativity'])

        # Split associativity values if they are combined (e.g., '2,8')
        associativity_values = [value.strip() for value in associativity.split(',')]

        if any(value in valid_associations for value in associativity_values):
            associated_foods = []
            for value in associativity_values:
                if value in associative_rules:
                    associated_food_items = recommended_foods[recommended_foods['Associativity'].isin(associative_rules[value])]
                    if not associated_food_items.empty:
                        associated_foods.append(associated_food_items['Food'].sample(1).values[0])

            if associated_foods:
                print(f"{food_item} (associated with: {', '.join(associated_foods)})")
            else:
                print(f"{food_item}")

# Function to ensure nutritional requirements
def ensure_nutritional_requirements(recommended_foods, target_nutrients):
    total_nutrients = recommended_foods[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']].sum()
    meets_requirements = all(total_nutrients >= pd.Series(target_nutrients))
    return meets_requirements

# Function to print recommendations
def print_recommendations(recommended_foods, meal_type, target_nutrients):
    print(f"\nRecommended {meal_type.capitalize()} Foods:")
    if ensure_nutritional_requirements(recommended_foods, target_nutrients):
        print_recommendations_with_associative_rules(recommended_foods, associativity_rules, ['0', '1', '3', '5', '6'])
    else:
        print(f"No {meal_type} recommendations meet the nutritional requirements.")

# Recommend food items for each meal type
print_recommendations(recommended_breakfast, 'breakfast', target_breakfast)
print_recommendations(recommended_lunch, 'lunch', target_lunch)
print_recommendations(recommended_dinner, 'dinner', target_dinner)
print_recommendations(recommended_snacks, 'snacks', target_snacks)
print_recommendations(recommended_appetizers, 'appetizers', target_appetizers)


In [ ]:
#prints the nutritional info
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import re

# Load the dataset
file_path = 'nutrition_cf.csv'
nutrition_data = pd.read_csv(file_path)

# Function to filter dataset based on multiple allergies, region, and category using regex
def filter_dataset(df, allergies, region_pattern, category):
    if allergies == ["no-allergies"]:
        allergy_condition = True
    else:
        allergy_condition = df['Allergy'].apply(lambda x: not any(re.search(allergy, x, re.IGNORECASE) for allergy in allergies))

    if category == 'veg':
        category_condition = df['Category'].str.lower() == 'veg'
    elif category == 'eggetarian':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg'])
    elif category == 'non-veg':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg', 'non-veg'])
    else:
        category_condition = False

    if region_pattern != "Null":
        region_filter = df['Region'].apply(lambda x: re.search(region_pattern, x, re.IGNORECASE) is not None)
    else:
        return df[allergy_condition & category_condition]

    return df[allergy_condition & region_filter & category_condition]

# Nutritional target values
target_breakfast = {
    'Proteins': 12,
    'Carbohydrates': 60,
    'Fats': 12,
    'Fiber': 6,
    'Energy(kcal)': 450
}

target_lunch = {
    'Proteins': 18,
    'Carbohydrates': 90,
    'Fats': 18,
    'Fiber': 9,
    'Energy(kcal)': 650
}

target_snacks = {
    'Proteins': 7,
    'Carbohydrates': 35,
    'Fats': 7,
    'Fiber': 3,
    'Energy(kcal)': 250
}

target_dinner = {
    'Proteins': 17,
    'Carbohydrates': 80,
    'Fats': 13,
    'Fiber': 8,
    'Energy(kcal)': 550
}

target_appetizers = {
    'Proteins': 5,
    'Carbohydrates': 30,
    'Fats': 5,
    'Fiber': 3,
    'Energy(kcal)': 200
}

# User inputs for allergies, region, and category
user_allergies_input = input("Enter allergies separated by commas (e.g., mushroom, peanut): ")
user_region_pattern = input("Enter region pattern (regex) (e.g., North): ")
user_category = input("Enter category (e.g., veg, non-veg, eggetarian): ")

print("User preference\nAllergy : "+user_allergies_input+"\nRegion : "+user_region_pattern+"\nCategory : "+user_category+"\n")

# Parsing user allergies input into a list
user_allergies = [allergy.strip() for allergy in user_allergies_input.split(',')]
if user_allergies == ['']:
    user_allergies = ["no-allergies"]

def recommend_food(df, meal_type, target_nutrients, num_recommendations=5, exclude_foods=[]):
    meal_data = df[df['Type'].str.contains(meal_type, case=False, na=False)]

    if exclude_foods:
        meal_data = meal_data[~meal_data['Food'].isin(exclude_foods)]

    features = ['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)', 'Carbon Footprint(kg CO2e)']

    X = meal_data[features].values
    y = meal_data['Food'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    knn = KNeighborsClassifier(n_neighbors=num_recommendations*2, metric='cosine')
    knn.fit(X_train_scaled, y_train)

    target_values = np.array([target_nutrients['Proteins'], target_nutrients['Carbohydrates'],
                              target_nutrients['Fats'], target_nutrients['Fiber'],
                              target_nutrients['Energy(kcal)'], 0]).reshape(1, -1)
    target_values_scaled = scaler.transform(target_values)

    distances, indices = knn.kneighbors(target_values_scaled)

    recommended_foods = meal_data.iloc[indices[0]].sample(n=num_recommendations)
    return recommended_foods

# Define associativity rules
associativity_rules = {
    '1': ['2'],
    '3': ['2', '4', '8'],
    '5': ['4', '8'],
    '6': ['7'],
    # Add more rules as needed
}

# Function to print recommendations with associative rules
def print_recommendations_with_associative_rules(recommended_foods, associative_rules, valid_associations):
    for index, row in recommended_foods.iterrows():
        food_item = row['Food']
        associativity = str(row['Associativity'])

        # Split associativity values if they are combined (e.g., '2,8')
        associativity_values = [value.strip() for value in associativity.split(',')]

        if any(value in valid_associations for value in associativity_values):
            associated_foods = []
            for value in associativity_values:
                if value in associative_rules:
                    associated_food_items = recommended_foods[recommended_foods['Associativity'].isin(associative_rules[value])]
                    if not associated_food_items.empty:
                        associated_foods.append(associated_food_items['Food'].sample(1).values[0])

            if associated_foods:
                print(f"{food_item} (associated with: {', '.join(associated_foods)})")
            else:
                print(f"{food_item}")

# Function to ensure nutritional requirements of combined foods
def ensure_combined_nutritional_requirements(recommended_foods, target_nutrients, associative_rules):
    combined_nutrients = recommended_foods[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']].sum()

    for index, row in recommended_foods.iterrows():
        associativity = str(row['Associativity']).split(',')
        for assoc in associativity:
            if assoc in associative_rules:
                associated_foods = recommended_foods[recommended_foods['Associativity'].isin(associative_rules[assoc])]
                combined_nutrients += associated_foods[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']].sum()

    meets_requirements = all(combined_nutrients >= pd.Series(target_nutrients))
    return meets_requirements

# Function to print nutritional information
def print_nutritional_information(food_items):
    print("Nutritional Information:")
    for index, row in food_items.iterrows():
        print(f"Food: {row['Food']}")
        print(f"Proteins: {row['Proteins']}g, Carbohydrates: {row['Carbohydrates']}g, Fats: {row['Fats']}g, Fiber: {row['Fiber']}g, Energy: {row['Energy(kcal)']} kcal")
        print("")

# Function to print recommendations
def print_recommendations(recommended_foods, meal_type, target_nutrients, associative_rules):
    print(f"\nRecommended {meal_type.capitalize()} Foods:")
    print_nutritional_information(recommended_foods)
    if ensure_combined_nutritional_requirements(recommended_foods, target_nutrients, associative_rules):
        print_recommendations_with_associative_rules(recommended_foods, associative_rules, ['0', '1', '3', '5', '6'])
    else:
        print(f"No {meal_type} recommendations meet the nutritional requirements.")

# Filtering the dataset based on user preferences
try:
    # Filtering the dataset based on user preferences
    filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)
    # Function to recommend food items for a specific meal type
    recommended_breakfast = recommend_food(filtered_data, 'Breakfast', target_breakfast)
    recommended_snacks = recommend_food(filtered_data, 'Snacks', target_snacks)
    recommended_appetizers = recommend_food(filtered_data, 'Appetizer', target_appetizers)

except:
    filtered_data = filter_dataset(nutrition_data, user_allergies, "Null", user_category)
    # Function to recommend food items for a specific meal type
    recommended_breakfast = recommend_food(filtered_data, 'Breakfast', target_breakfast)
    recommended_snacks = recommend_food(filtered_data, 'Snacks', target_snacks)
    recommended_appetizers = recommend_food(filtered_data, 'Appetizer', target_appetizers)
finally:
    filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)
    recommended_lunch = recommend_food(filtered_data, 'Lunch', target_lunch)
    recommended_dinner = recommend_food(filtered_data, 'Dinner', target_dinner)

# Recommend food items for each meal type
print_recommendations(recommended_breakfast, 'breakfast', target_breakfast, associativity_rules)
print_recommendations(recommended_lunch, 'lunch', target_lunch, associativity_rules)
print_recommendations(recommended_dinner, 'dinner', target_dinner, associativity_rules)
print_recommendations(recommended_snacks, 'snacks', target_snacks, associativity_rules)
print_recommendations(recommended_appetizers, 'appetizers', target_appetizers, associativity_rules)


In [ ]:
#this kinda works for associative nutritional requirements
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import re

# Load the dataset
file_path = 'nutrition_cf.csv'
nutrition_data = pd.read_csv(file_path)

# Function to filter dataset based on multiple allergies, region, and category using regex
def filter_dataset(df, allergies, region_pattern, category):
    if allergies == ["no-allergies"]:
        allergy_condition = True
    else:
        allergy_condition = df['Allergy'].apply(lambda x: not any(re.search(allergy, x, re.IGNORECASE) for allergy in allergies))

    if category == 'veg':
        category_condition = df['Category'].str.lower() == 'veg'
    elif category == 'eggetarian':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg'])
    elif category == 'non-veg':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg', 'non-veg'])
    else:
        category_condition = False

    if region_pattern != "Null":
        region_filter = df['Region'].apply(lambda x: re.search(region_pattern, x, re.IGNORECASE) is not None)
    else:
        return df[allergy_condition & category_condition]

    return df[allergy_condition & region_filter & category_condition]

# Nutritional target values
target_breakfast = {
    'Proteins': 12,
    'Carbohydrates': 60,
    'Fats': 12,
    'Fiber': 6,
    'Energy(kcal)': 450
}

target_lunch = {
    'Proteins': 13,
    'Carbohydrates': 60,
    'Fats': 13,
    'Fiber': 6,
    'Energy(kcal)': 450
}

target_appetizers = {
    'Proteins': 5,
    'Carbohydrates': 30,
    'Fats': 5,
    'Fiber': 3,
    'Energy(kcal)': 200
}

target_snacks = {
    'Proteins': 7,
    'Carbohydrates': 35,
    'Fats': 7,
    'Fiber': 3,
    'Energy(kcal)': 250
}

target_dinner = {
    'Proteins': 17,
    'Carbohydrates': 80,
    'Fats': 13,
    'Fiber': 8,
    'Energy(kcal)': 550
}


# User inputs for allergies, region, and category
user_allergies_input = input("Enter allergies separated by commas (e.g., mushroom, peanut): ")
user_region_pattern = input("Enter region pattern (regex) (e.g., North): ")
user_category = input("Enter category (e.g., veg, non-veg, eggetarian): ")

print("User preference\nAllergy : "+user_allergies_input+"\nRegion : "+user_region_pattern+"\nCategory : "+user_category+"\n")

# Parsing user allergies input into a list
user_allergies = [allergy.strip() for allergy in user_allergies_input.split(',')]
if user_allergies == ['']:
    user_allergies = ["no-allergies"]

def recommend_food(df, meal_type, target_nutrients, num_recommendations=5, exclude_foods=[]):
    meal_data = df[df['Type'].str.contains(meal_type, case=False, na=False)]

    if exclude_foods:
        meal_data = meal_data[~meal_data['Food'].isin(exclude_foods)]

    features = ['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)', 'Carbon Footprint(kg CO2e)']

    X = meal_data[features].values
    y = meal_data['Food'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    knn = KNeighborsClassifier(n_neighbors=num_recommendations*2, metric='cosine')
    knn.fit(X_train_scaled, y_train)

    target_values = np.array([target_nutrients['Proteins'], target_nutrients['Carbohydrates'],
                              target_nutrients['Fats'], target_nutrients['Fiber'],
                              target_nutrients['Energy(kcal)'], 0]).reshape(1, -1)
    target_values_scaled = scaler.transform(target_values)

    distances, indices = knn.kneighbors(target_values_scaled)

    recommended_foods = meal_data.iloc[indices[0]].sample(n=num_recommendations)
    return recommended_foods

# Define associativity rules
associativity_rules = {
    '1': ['2'],
    '3': ['2', '4', '8'],
    '5': ['4', '8'],
    '6': ['7'],
    # Add more rules as needed
}

# Function to print recommendations with associative rules
def print_recommendations_with_associative_rules(recommended_foods, associative_rules):
    print("Recommended Foods with Associativity:")
    for index, row in recommended_foods.iterrows():
        food_item = row['Food']
        associativity = str(row['Associativity'])

        # Split associativity values if they are combined (e.g., '2,8')
        associativity_values = [value.strip() for value in associativity.split(',')]

        associated_foods = []
        for value in associativity_values:
            if value in associative_rules:
                associated_food_items = recommended_foods[recommended_foods['Associativity'].isin(associative_rules[value])]
                if not associated_food_items.empty:
                    associated_foods.append(associated_food_items['Food'].sample(1).values[0])

        if associated_foods:
            print(f"{food_item} (associated with: {', '.join(associated_foods)})")
        else:
            print(f"{food_item}")

# Function to check nutritional requirements for combined associated foods
def check_combined_nutritional_requirements(food1, food2, target_nutrients):
    combined_nutrients = food1[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']] + food2[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']]
    meets_requirements = all(combined_nutrients >= pd.Series(target_nutrients))
    return meets_requirements

# Function to print nutritional information
'''def print_nutritional_information(food_items):
    print("Nutritional Information:")
    for index, row in food_items.iterrows():
        print(f"Food: {row['Food']}")
        print(f"Proteins: {row['Proteins']}g, Carbohydrates: {row['Carbohydrates']}g, Fats: {row['Fats']}g, Fiber: {row['Fiber']}g, Energy: {row['Energy(kcal)']} kcal")
        print("")'''

# Function to print recommendations
def print_recommendations(recommended_foods, meal_type, target_nutrients, associative_rules):
    print(f"\nRecommended {meal_type.capitalize()} Foods:")
    #print_nutritional_information(recommended_foods)
    valid_associations = ['0', '1', '3', '5', '6']

    for index, row in recommended_foods.iterrows():
        food_item = row['Food']
        associativity = str(row['Associativity'])

        # Split associativity values if they are combined (e.g., '2,8')
        associativity_values = [value.strip() for value in associativity.split(',')]

        for value in associativity_values:
            if value in valid_associations and value in associative_rules:
                associated_food_items = recommended_foods[recommended_foods['Associativity'].isin(associative_rules[value])]
                if not associated_food_items.empty:
                    for _, assoc_row in associated_food_items.iterrows():
                        if check_combined_nutritional_requirements(row, assoc_row, target_nutrients):
                            print(f"{food_item} (associated with: {assoc_row['Food']})")
                            break
        else:
            print(f"{food_item}")

# Filtering the dataset based on user preferences
try:
    # Filtering the dataset based on user preferences
    filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)
    # Function to recommend food items for a specific meal type
    recommended_breakfast = recommend_food(filtered_data, 'Breakfast', target_breakfast)
    recommended_snacks = recommend_food(filtered_data, 'Snacks', target_snacks)
    recommended_appetizers = recommend_food(filtered_data, 'Appetizer', target_appetizers)

except:
    filtered_data = filter_dataset(nutrition_data, user_allergies, "Null", user_category)
    # Function to recommend food items for a specific meal type
    recommended_breakfast = recommend_food(filtered_data, 'Breakfast', target_breakfast)
    recommended_snacks = recommend_food(filtered_data, 'Snacks', target_snacks)
    recommended_appetizers = recommend_food(filtered_data, 'Appetizer', target_appetizers)
finally:
    filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)
    recommended_lunch = recommend_food(filtered_data, 'Lunch', target_lunch)
    recommended_dinner = recommend_food(filtered_data, 'Dinner', target_dinner)

# Recommend food items for each meal type
print_recommendations(recommended_breakfast, 'breakfast', target_breakfast, associativity_rules)
print_recommendations(recommended_lunch, 'lunch', target_lunch, associativity_rules)
print_recommendations(recommended_dinner, 'dinner', target_dinner, associativity_rules)
print_recommendations(recommended_snacks, 'snacks', target_snacks, associativity_rules)
print_recommendations(recommended_appetizers, 'appetizers', target_appetizers, associativity_rules)
